In [8]:
import pandas as pd
import math
import random

def convert_speed(row):
    KNOTPERMETER = 1.94
    PI = 3.1416
    dir_x = 0
    dir_y = 0
    deg = row['course']
    if (deg >= 0 or deg == 360) and deg < 90:
        if deg == 0 or deg == 360:
            dir_x = 0
        else:
            dir_x = 1
        dir_y = 1
    elif deg >= 90 and deg < 180:
        if deg == 90:
            dir_y = 0
        else:
            dir_y = -1
        dir_x = 1
    elif deg >= 180 and deg < 270:
        if deg == 180:
            dir_x = 0
        else:
            dir_x = -1
        dir_y = -1
    elif deg >= 270 and deg < 360:
        if deg == 270:
            dir_y = 0
        else:
            dir_y = 1
        dir_x = -1
    rad = deg * PI / 180
    vx = round(row['speed'] * math.cos(rad) * KNOTPERMETER, 5)
    vy = round(row['speed'] * math.sin(rad) * KNOTPERMETER, 5)
    return str(vx)+','+str(vy)

def convert_loc(row):  #convert longlat to meter/km
    LONPERMETER = 111139
    row['lon'] = row['lon'] * LONPERMETER
    row['lat'] = row['lat'] * LONPERMETER
    return row

def combine_longlat(row):  #convert longlat to meter/km
    return str(row['lon'])+','+str(row['lat'])

def random_radius(row):
    SCALE_CONST = 1
    MIN = 400
    MAX = 600
    new_rad = float(random.randint(MIN, MAX))/SCALE_CONST
    return str(new_rad)

## Get Obj Data

In [9]:
df = pd.read_csv('ais.csv')
df = df.dropna()
df['id'] = pd.factorize(df['shipid'])[0]
df.head()

,t,shipid,lon,lat,heading,course,speed,shiptype,destination,id
0,2020-02-29T22:00:01Z,10807db4-2c77-4a7a-a418-0af8f0d672d8,23.537350,37.856717,40.0,0.0,0.0,70.0,GRPIR,0
1,2020-02-29T22:00:01Z,203d4b0c-6808-454b-ae7a-382dbd3d79e3,23.641017,37.948300,181.0,349.9,0.0,89.0,PIRAEUS,1
3,2020-02-29T22:00:01Z,8b998a42-caba-4baa-81e2-2318f290906e,23.668633,37.938840,296.0,0.1,0.0,37.0,AEGEAN SEA,2
5,2020-02-29T22:00:02Z,642d0bd7-519c-42d5-a294-215d889da3d1,23.547662,37.913112,344.0,33.3,0.1,80.0,PERIOCHI,3
10,2020-02-29T22:00:03Z,076431cc-5ae5-4a0f-a46c-726b56443ca5,23.683820,37.928758,23.0,248.2,0.0,37.0,PROVA,4


In [10]:
df['t'] = pd.factorize(df['t'])[0]
df['v'] = df.apply(lambda row: convert_speed(row), axis=1)
df['r'] = df.apply(lambda row: random_radius(row), axis=1)
df['location'] = df.apply(lambda row: combine_longlat(row), axis=1)
df = df.apply(lambda row: convert_loc(row), axis=1)

del df['speed']
del df['course']
del df['shiptype']
del df['destination']
del df['heading']
del df['shipid']
del df['lon']
del df['lat']
df.head()

,t,lon,lat,id,v,r,location
0,0,2.615918e+06,4.207358e+06,0,"0.0,0.0",475.0,"23.53735,37.85671666666666"
1,0,2.627439e+06,4.217536e+06,1,"0.0,-0.0",469.0,"23.641016666666665,37.9483"
3,0,2.630508e+06,4.216485e+06,2,"0.0,0.0",588.0,"23.66863333333333,37.93884"
5,1,2.617064e+06,4.213625e+06,3,"0.16215,0.10651",423.0,"23.547661666666666,37.913111666666666"
10,2,2.632196e+06,4.215364e+06,4,"-0.0,-0.0",452.0,"23.68382,37.92875833333333"


In [11]:
del df['lon']
del df['lat']

In [12]:
neworder = ['t','id','location','v', 'r']
df=df.reindex(columns=neworder)
df.head()

,t,id,location,v,r
0,0,0,"23.53735,37.85671666666666","0.0,0.0",475.0
1,0,1,"23.641016666666665,37.9483","0.0,-0.0",469.0
3,0,2,"23.66863333333333,37.93884","0.0,0.0",588.0
5,1,3,"23.547661666666666,37.913111666666666","0.16215,0.10651",423.0
10,2,4,"23.68382,37.92875833333333","-0.0,-0.0",452.0


In [13]:
filename = 'events_AIS.csv'
df.to_csv(filename, index=False, header=False, sep='|')